In [121]:
import cifar10,cifar10_input  
import tensorflow as tf  
import numpy as np  
import time  

In [122]:
max_steps = 3000  
batch_size = 128  
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin' 

In [123]:
def variable_with_weight_loss(shape, stddev, wl):  
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))  
    if wl is not None:  
        weight_loss = tf.multiply(tf.nn.l2_loss(var), wl, name='weight_loss')  
        tf.add_to_collection('losses', weight_loss)  
    return var  

In [124]:
def loss(logits, labels):  
    labels = tf.cast(labels, tf.int64)  
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(  
        logits=logits, labels=labels, name='cross_entropy_per_example')  
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')  
    tf.add_to_collection('losses', cross_entropy_mean)  
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [125]:
cifar10.maybe_download_and_extract() 

In [126]:
images_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir,  
                                                            batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [127]:
images_test, labels_test = cifar10_input.inputs(eval_data=True,  
                                                data_dir=data_dir,  
                                                batch_size=batch_size)

In [128]:
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])  
label_holder = tf.placeholder(tf.int32, [batch_size]) 

In [129]:
weight1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=5e-2, wl=0.0)  
kernel1 = tf.nn.conv2d(image_holder, weight1, [1, 1, 1, 1], padding='SAME')  
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))  
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))  
pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],  
                       padding='SAME')  
norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75) 

In [130]:
weight2 = variable_with_weight_loss(shape=[5, 5, 64, 64], stddev=5e-2, wl=0.0)  
kernel2 = tf.nn.conv2d(norm1, weight2, [1, 1, 1, 1], padding='SAME')  
bias2 = tf.Variable(tf.constant(0.1, shape=[64]))  
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))  
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)  
pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],  
                       padding='SAME')  

In [131]:
reshape = tf.reshape(pool2, [batch_size, -1])  
dim = reshape.get_shape()[1].value  
weight3 = variable_with_weight_loss(shape=[dim, 384], stddev=0.04, wl=0.004)  
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))  
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)  

In [132]:
weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04, wl=0.004)  
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))                                        
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)  

In [133]:
weight5 = variable_with_weight_loss(shape=[192, 10], stddev=1/192.0, wl=0.0)  
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))  
logits = tf.add(tf.matmul(local4, weight5), bias5)  

In [134]:
loss = loss(logits, label_holder)  

In [135]:
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss) 

In [136]:
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

In [137]:
sess = tf.InteractiveSession()  
tf.global_variables_initializer().run()  

In [138]:
tf.train.start_queue_runners() 

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 123148281806848)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123148287062016)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123148292317184)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123148297572352)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123148302827520)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123148308082688)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123148313337856)>,
 <Thread(QueueRunnerThread-shuffle_batch/

In [139]:
for step in range(max_steps):  
    start_time = time.time()  
    image_batch,label_batch = sess.run([images_train,labels_train])  
    _, loss_value = sess.run([train_op, loss],feed_dict={image_holder: image_batch,   
                                                         label_holder:label_batch})  
    duration = time.time() - start_time  
  
    if step % 10 == 0:  
        examples_per_sec = batch_size / duration  
        sec_per_batch = float(duration)  
      
        format_str = ('step %d, loss = %.2f (%.1f examples/sec; %.3f sec/batch)')  
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch)) 

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_9' with dtype int32 and shape [128]
	 [[Node: Placeholder_9 = Placeholder[dtype=DT_INT32, shape=[128], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_9', defined at:
  File "/Users/wanhua/anaconda/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/wanhua/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/wanhua/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/wanhua/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/wanhua/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/wanhua/anaconda/lib/python3.5/site-packages/zmq/eventloop/ioloop.p

In [ ]:
num_examples = 10000  
import math  
num_iter = int(math.ceil(num_examples / batch_size))  
true_count = 0    
total_sample_count = num_iter * batch_size  
step = 0  
while step < num_iter:  
    image_batch,label_batch = sess.run([images_test,labels_test])  
    predictions = sess.run([top_k_op],feed_dict={image_holder: image_batch,  
                                                 label_holder:label_batch})  
    true_count += np.sum(predictions)  
    step += 1  

In [ ]:
precision = true_count / total_sample_count  
print('precision @ 1 = %.3f' % precision)